In [1]:
from transformers import pipeline
import textstat
import gradio as gr

In [ ]:
# You can switch to 't5-small' or others if needed
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [8]:
def summarize_article(text):
    max_chunk = 1024
    text = text.replace('\n', ' ')
    inputs = [text[i:i+max_chunk] for i in range(0, len(text), max_chunk)]

    summary = ''
    for chunk in inputs:
        out = summarizer(chunk, max_length=150, min_length=40, do_sample=False)
        summary += out[0]['summary_text'] + ' '
    
    summary = summary.strip()
    
    stats = {
        "Original Word Count": len(text.split()),
        "Summary Word Count": len(summary.split()),
        "Readability Score (Flesch)": round(textstat.flesch_reading_ease(summary), 2)
    }
    return summary, stats


In [ ]:
def gradio_interface(article_text):
    summary, stats = summarize_article(article_text)
    stats_text = f"""
    - Original Word Count: {stats['Original Word Count']}
    - Summary Word Count: {stats['Summary Word Count']}
    - Readability Score: {stats['Readability Score (Flesch)']}
    """
    return summary, stats_text

demo = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=15, label="Paste your article here"),
    outputs=[
        gr.Textbox(label="Generated Summary"),
        gr.Textbox(label="Summary Stats")
    ],
    title="📝 AI Writer & Summarizer",
    description="Enter a long article, and this app will return a concise summary with readability metrics.",
    theme="default"
)

demo.launch(inline=True, share=True)